# Imports

In [1]:
# Python packages
import sys
sys.path.append('../../')
from datetime import datetime
import numpy as np
import pandas as pd
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV, cross_validate, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor, XGBClassifier

# Custom functions
import src.settings as settings
import src.mapper_cols as mapper_cols
from src.run_all.main_get_data import get_data
from src.run_all.main_preprocess import preprocess_data
from src.utilities.utilities import get_latest_file, list_filenames

# instellingen voor panda weergave aanpassen
pd.set_option('display.max_rows', 500) # alle rijen tonen
pd.set_option('display.max_columns', 500) # alle kolommen tonen
pd.set_option('display.width', 1000) # kolombreedte
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.15f}'.format(x)) # floats output tot 15 decimalen

### Settings

In [2]:
# Location all data
datapath = '../../data/'
# Drop target values in X set
X_DROP_VALUES = settings.Y_TARGET_COLS
# de kolom die wordt gebruikt als y value
Y_VALUE = ['wmoclientenper1000inwoners']
# test size voor de train/test split
TEST_SIZE = 0.3
# random state voor de train/test split. Bijvoorbeeld random_state = 42 als vaste seed voor reproduceerbaarheid
RANDOM_STATE = 42

## Model parameters
# manier van cross validate in de modellen. Bijvoorbeeld 10 of RepeatedKFold(n_splits=30, n_repeats=5, random_state=1)
CROSS_VALIDATE = 5
# manier van scoren in de modellen
MODEL_SCORING = 'neg_mean_squared_error'
## Grid Search parameters

# parameters die gebruikt worden in de grid search
ALPHA = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
NEIGHBORS = [3, 5, 11, 19]
NORMALIZE = [True, False]
KERNEL = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
GAMMA = [0.5, 1, 1.5, 2, 5]
N_ESTIMATORS = [50,100,200]
C_REGULARIZATION = [0.001, 0.01, 0,1, 1]

### Functions

In [3]:
def drop_nan_from_specific_columns (df,columns_to_check):
    """
    Drops all rows with nan values in specific columns in a dataframe
    """
    df.dropna(
        axis=0,
        how='any',
        thresh=None,
        subset=columns_to_check,
        inplace=True
    )
    
def split_clf_and_params(best_estimator_clf):
    """
    Takes best estimator[clf] and outputs a list with clf and parameters
    """
    clf_and_params = str(best_estimator_clf)
    clf_and_params = clf_and_params.replace(")", "")
    clf_and_params_split = clf_and_params.split("(")
    return clf_and_params_split

def rmse_from_neg_mean_squared_error(neg_mean_squared_error):
    """
    Calculates RMSE from the neq mean squared error
    """
    rmse = np.sqrt(-(neg_mean_squared_error))
    return (rmse)

# functie maken om op basis van de grid search best estimator, het beste RMSE model te selecteren 
def rmse_from_gridsearch_best_estimator(grid_search):
    """
    Calculates RMSE from the grid search best estimator
    """
    rmse = np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test)))
    return (rmse)

# Get data
This step will load and combine several tables from CBS statline. 

Note: This step takes a number of minutes and without changes to the settings will give the same result. Therefor this code is commented out and the original dataset is loaded. 

In [4]:
%%time 
# ## CREATE NEW DATASET
# df_get_data_WMO= get_data(save_all=True)

# ## HARDCODED
datapath = '../../data/'
filename = 'df_get_data_WMO_WIJK_HUISHOUDENS_BEVOLKING_HEFFING_202104042111.parquet.gzip'
df_get_data_WMO = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
# datapath = '../../data/'
# df = get_latest_file(filename_str_contains='df_WMO_', datapath=datapath, filetype='parquet')

print(f"The shape of the dataframe from step 'Get Data': {df_get_data_WMO.shape}")
df_get_data_WMO.sample(5)

The shape of the dataframe from step 'Get Data': (936, 470)
CPU times: user 153 ms, sys: 61.6 ms, total: 214 ms
Wall time: 226 ms


,,perioden,typemaatwerkarrangement,wmoclienten,wmoclientenper1000inwoners,aantalinkomensontvangers,aantalinwoners,actieven1575jaar,afstandtotgrotesupermarkt,afstandtothuisartsenpraktijk,afstandtotkinderdagverblijf,afstandtotschool,alandbouwbosbouwenvisserij,bedrijfsmotorvoertuigen,bedrijfsvestigingentotaal,bevolkingsdichtheid,bfnijverheidenenergie,bouwjaarvanaf2000,bouwjaarvoor2000,dekkingspercentage,eenpersoonshuishoudens,eigendomonbekend,geboorterelatief,geboortetotaal,gehuwd,gemeentenaam,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruikeigenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemgasverbruikappartement,gemgasverbruikeigenwoning,gemgasverbruikhoekwoning,gemgasverbruikhuurwoning,gemgasverbruiktussenwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgestandaardiseerdinkomenvanhuish,gemiddeldaardgasverbruiktotaal,gemiddeldehuishoudensgrootte,gemiddeldelektriciteitsverbruiktotaal,gemiddeldewoningwaarde,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,gescheiden,geweldsenseksuelemisdrijven,gihandelenhoreca,hjvervoerinformatieencommunicatie,huishonderofrondsociaalminimum,huishoudensmeteenlaaginkomen,huishoudensmetkinderen,huishoudenstot110vansociaalminimum,huishoudenstot120vansociaalminimum,huishoudenstotaal,huishoudenszonderkinderen,huurwoningentotaal,inbezitoverigeverhuurders,inbezitwoningcorporatie,jongerenmetjeugdzorginnatura,k0tot15jaar,k15tot25jaar,k20huishoudensmethoogsteinkomen,k20personenmethoogsteinkomen,k25tot45jaar,k40huishoudensmetlaagsteinkomen,k40personenmetlaagsteinkomen,k45tot65jaar,k65jaarofouder,klfinancieledienstenonroerendgoed,koopwoningen,mannen,marokko,matevanstedelijkheid,mediaanvermogenvanparticulierehuish,meestvoorkomendepostcode,mnzakelijkedienstverlening,motorfietsen,nederlandseantillenenaruba,nettoarbeidsparticipatie,nietwesterstotaal,omgevingsadressendichtheid,ongehuwd,opleidingsniveauhoog,opleidingsniveaulaag,opleidingsniveaumiddelbaar,oppervlakteland,oppervlaktetotaal,oppervlaktewater,overignietwesters,percentagebewoond,percentageeengezinswoning,percentagejongerenmetjeugdzorg,percentagemeergezinswoning,percentageonbewoond,percentagewerknemers,percentagewoningenmetstadsverwarming,percentagezelfstandigen,personenautos6jaarenouder,personenautosbrandstofbenzine,personenautosjongerdan6jaar,personenautosnaaroppervlakte,personenautosoverigebrandstof,personenautosperhuishouden,personenautostotaal,personenpersoortuitkeringao,personenpersoortuitkeringaow,personenpersoortuitkeringbijstand,personenpersoortuitkeringww,rucultuurrecreatieoverigediensten,scholenbinnen3km,sterfterelatief,sterftetotaal,suriname,totaaldiefstaluitwoningschuured,turkije,vernielingmisdrijftegenopenbareorde,verweduwd,vrouwen,westerstotaal,woningvoorraad,alleenstaande_mannen,alleenstaande_totaal_mannen_en_vrouwen,alleenstaande_vrouwen,ouder_in_eenouderhuishouden_mannen,ouder_in_eenouderhuishouden_totaal_mannen_en_vrouwen,ouder_in_eenouderhuishouden_vrouwen,overig_lid_huishouden_mannen,overig_lid_huishouden_totaal_mannen_en_vrouwen,overig_lid_huishouden_vrouwen,"partner_in_paar,_geen_kind(eren)_mannen","partner_in_paar,_geen_kind(eren)_totaal_mannen_en_vrouwen","partner_in_paar,_geen_kind(eren)_vrouwen",partner_in_paar_met_kind(eren)_mannen,partner_in_paar_met_kind(eren)_totaal_mannen_en_vrouwen,partner_in_paar_met_kind(eren)_vrouwen,persoon_in_institutioneel_huishouden_mannen,persoon_in_institutioneel_huishouden_totaal_mannen_en_vrouwen,persoon_in_institutioneel_huishouden_vrouwen,persoon_in_particulier_huishouden_mannen,persoon_in_particulier_huishouden_totaal_mannen_en_vrouwen,persoon_in_particulier_huishouden_vrouwen,thuiswonend_kind_mannen,thuiswonend_kind_totaal_mannen_en_vrouwen,thuiswonend_kind_vrouwen,totaal_personen_mannen,totaal_personen_totaal_mannen_en_vrouwen,totaal_personen_vrouwen,popaantalbasisonderwijs

# Preprocess --> Create test sets
This step will transform (select columns, impute, scale) the dataframe to be used in train/predict. 

In [5]:
# %%time 
# df_preprocessed = preprocess_data(df=df_get_data_WMO, save_all=True)

# print(f"The shape of the dataframe from step 'Preprocess': {df_preprocessed.shape}")
# df_preprocessed.sample(5)

In [6]:
# # For testing train
datapath = '../../data/'
# filename = 'df_preprocessed_202104042151_All.parquet.gzip'
filename = 'df_preprocessed_202104042151_Boerenverstand_Maikel.parquet.gzip'
# filename = 'df_preprocessed_202104042151_Minimum_Maikel.parquet.gzip'
# filename = 'df_preprocessed_202104042151_No_Relative.parquet.gzip'
df_preprocessed = pd.read_parquet(datapath + filename)
input_filename = filename

In [19]:
df_preprocessed.head()

,,wmoclienten,wmoclientenper1000inwoners,aantalinwoners,gemiddeldehuishoudensgrootte,gescheiden,verweduwd,alleenstaande_mannen,alleenstaande_vrouwen,ouder_in_eenouderhuishouden_mannen,ouder_in_eenouderhuishouden_vrouwen,popaantalrestaurantsbinnen3km,popafstandtothuisartsenpraktijk,poparbeidsongeschiktheidtotaal,popbevolkingsdichtheid,popeenpersoonshuishoudensrelatief,popk65tot80jaarrelatieveleeftijdsgroep,popk80jaarofouderrelatieveleeftijdsgroep,popomgevingsadressendichtheid,poppersonenautosrelatief,popwerkloosheid,relative_mannen,relative_vrouwen,relative_alleenstaande_mannen,relative_alleenstaande_vrouwen,relative_gescheiden,relative_ongehuwd,relative_ouder_in_eenouderhuishouden_vrouwen,relative_ouder_in_eenouderhuishouden_mannen,relative_eenpersoonshuishoudens,relative_huishoudenszonderkinderen,relative_huishoudensmetkinderen,relative_popaantalrestaurantsbinnen3km,relative_popafstandtothuisartsenpraktijk,relative_poparbeidsongeschiktheidtotaal,relative_popbevolkingsdichtheid,relative_popk65tot80jaarrelatieveleeftijdsgroep,relative_popk80jaarofouderrelatieveleeftijdsgroep,relative_popomgevingsadressendichtheid,relative_popsterkstedelijk,relative_popmatigstedelijk,relative_popweinigstedelijk,relative_popnietstedelijk,relative_poptotaleoppervlakte
codering_regio,interval,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GM1680,2019,280.000000000000000,11.000000000000000,0.028367823505597,0.352941176470588,0.023919649339823,0.056821469907407,0.002698602712018,0.003476586526890,0.007500604887491,0.003295025315438,0.010270149586961,0.833333333333333,0.031405928969768,0.012152166255724,0.238663484486873,0.908450704225352,0.602941176470588,0.025307912940779,0.296404275996113,0.018951358180670,0.498660679114473,0.501339320885527,0.107736547703459,0.135113842275270,0.072638462144489,0.397699519420153,0.033483022138186,0.012211455132750,0.291786226685796,0.388809182209469,0.319404591104735,0.000200898132829,0.000094540297802,0.042937051918380,0.003624044749074,0.000791774994091,0.000232411565430,0.011029701410226,0.000000000000000,0.000000000000000,0.163082013708343,0.837075553454660,0.010985188686678
GM0197,2019,1080.000000000000000,40.000000000000000,0.030252902151078,0.411764705882353,0.019334713744050,0.059317129629630,0.011408433262471,0.013494504318990,0.026010162109848,0.012979185083983,0.014958696137531,0.375000000000000,0.032873495744056,0.045262416343783,0.248210023866348,0.669014084507042,0.529411764705882,0.110005061582588,0.245869776482021,0.013897662665824,0.501980674540002,0.498019325459998,0.413720336159343,0.473695901669690,0.055643996890156,0.444004294546666,0.120691570101070,0.039798600570138,0.295639968418282,0.345556627774366,0.358803403807352,0.000266558068935,0.000048128540224,0.042205027581356,0.010366147125245,0.000618266632113,0.000199918551701,0.028951168042649,0.033689978157047,0.316167487319981,0.358742734441524,0.292103217207804,0.003592980637518
GM0059,2019,900.000000000000000,32.000000000000000,0.031228502853139,0.470588235294118,0.023449054777207,0.054904513888889,0.003519019215290,0.003546118257428,0.009920154851198,0.004862171502049,0.006251395400759,0.500000000000000,0.033754035808629,0.043853469531525,0.260143198090692,0.570422535211268,0.367647058823529,0.050109667622743,0.254616132167153,0.027795325331649,0.508652879505960,0.491347120494040,0.126741347120494,0.125484704868591,0.064950452391211,0.452642539135430,0.044521039781703,0.014720666379434,0.301442348724076,0.314585644789622,0.383972006486302,0.000118483412322,0.000057446502944,0.042007755277897,0.009765905500503,0.000549332184403,0.000154387476662,0.015331035473216,0.000000000000000,0.000000000000000,0.438388625592417,0.561539566278903,0.003733304610082
GM0482,2019,590.000000000000000,29.000000000000000,0.022199846177583,0.470588235294118,0.016027106246807,0.038628472222222,0.009024581535985,0.012793838418955,0.021292039680619,0.015912561279434,0.021656619781201,0.166666666666667,0.020252421485178,0.398555829517436,0.255369928400955,0.5140845070

In [7]:
df = df_preprocessed.copy()

# Train

## Train model

#### Stappen hieronder mogelijk verplaatsten naar prepare stap, later beoordelen

In [8]:
# checken of er rijen in het dataframe zitten waarbij de Y_value leeg is. Die rijen worden eruit gehaald.
drop_nan_from_specific_columns(df,Y_VALUE)

In [9]:
# X en y aanmaken
X = df.drop(X_DROP_VALUES, axis=1)
y = df[Y_VALUE]
# splitsen van X en y in train/test. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)

## Gridsearch

In [10]:
# pipeline maken om in de grid search te kunnen gebruiken
pl_gs_total = Pipeline([('clf', LinearRegression())]) # Placeholder Estimator
    
# param grid waarin alle classifiers + hyper parameters kunnen worden opgenomen. 
# hier classifiers (modellen) + parameters toevoegen
param_grid_total = [{'clf': [LinearRegression()], 
                     'clf__normalize': NORMALIZE,},
                    
                    {'clf': [Ridge()],  
                     'clf__alpha': ALPHA},
                    
                    {'clf': [Lasso()], 
                     'clf__alpha': ALPHA},
                   
                    {'clf': [KNeighborsRegressor()],  
                     'clf__n_neighbors': NEIGHBORS},
                     
                  # {'clf': [SVR()], 
                  #  'clf__kernel': KERNEL,
                  #  'clf__C': C_REGULARIZATION},
                    
                   {'clf': [XGBRegressor()],  
                    'clf__gamma': GAMMA,
                    'clf__n_estimators': N_ESTIMATORS},                   
                   ]
    
# grid search aanmaken
grid_search_total = GridSearchCV(pl_gs_total, param_grid_total, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)

In [11]:
%%time
#grid search uitvoeren
grid_search_total.fit(X_train, y_train)

/usr/local/Caskroom/miniconda/base/envs/jads-env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.8260561100141786, tolerance: 3.3415973501005385
  model = cd_fast.enet_coordinate_descent(
/usr/local/Caskroom/miniconda/base/envs/jads-env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12.237970504374971, tolerance: 3.4077536627760523
  model = cd_fast.enet_coordinate_descent(
/usr/local/Caskroom/miniconda/base/envs/jads-env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31.870215983733942, tolerance: 3.4341040431183036
  model = cd_fast.enet_coordinate_descent(


CPU times: user 1min 53s, sys: 1.17 s, total: 1min 54s
Wall time: 14.8 s


GridSearchCV(cv=5, estimator=Pipeline(steps=[('clf', LinearRegression())]),
             param_grid=[{'clf': [LinearRegression()],
                          'clf__normalize': [True, False]},
                         {'clf': [Ridge()],
                          'clf__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
                         {'clf': [Lasso()],
                          'clf__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
                         {'clf': [KNeighborsRegressor()],
                          'clf__n_neighbors': [3, 5, 11, 19]},
                         {'clf': [XGBRegresso...
                                               missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=50, n_jobs=None,
                                               num_parallel_tree=None,
                                               random_state=None,
                                             

### Evaluate

In [12]:
# de best estimator uit de grid search halen (beste train score)
print(f"Het model met de beste train score is:\n{grid_search_total.best_estimator_['clf']}")
# de RMSE berekenen voor de best estimator
print(f"Dit model heeft een train score RMSE van {rmse_from_neg_mean_squared_error(grid_search_total.best_score_)}") 
print(f"Dit model heeft een test score RMSE van  {rmse_from_gridsearch_best_estimator(grid_search_total)}")

Het model met de beste train score is:
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
Dit model heeft een train score RMSE van 5.4440744371876235
Dit model heeft een test score RMSE van  5.2714100899898275


### Save best model and best model properties

In [13]:
# opslaan van beste estimator vanuit de gridsearch naar een Pickle file
suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')
output_filename = f'../../data/best_model_{suffix_datetime}.pickle'
pickle.dump(grid_search_total.best_estimator_, open(output_filename, 'wb'))

In [14]:
# extra regel om tijdelijk een dummy bij input_filename te krijgen
# input_filename = 'Hier komt uiteindelijk de input_filename_locatie'
# dictionary maken van alle properties die van het beste model moeten worden opgeslagen
best_model_properties_dict = {"Model": [split_clf_and_params(grid_search_total.best_estimator_['clf'])[0]],
                        "Gridsearch_Params": [split_clf_and_params(grid_search_total.best_estimator_['clf'])[1]],
                        "Train_RMSE": [rmse_from_neg_mean_squared_error(grid_search_total.best_score_)],
                        "Test_RMSE": [rmse_from_gridsearch_best_estimator(grid_search_total)],
                        "Number_of_features": [len(X.columns)],
                        "Y_value": Y_VALUE,
                        "Input_filename": [input_filename],
                        "Output_filename": [output_filename],
                                     }
best_model_properties = pd.DataFrame(best_model_properties_dict)

In [15]:
best_model_properties

,Model,Gridsearch_Params,Train_RMSE,Test_RMSE,Number_of_features,Y_value,Input_filename,Output_filename
0,XGBRegressor,"base_score=0.5, booster='gbtree', colsample_by...",5.444074437187624,5.271410089989828,41,wmoclientenper1000inwoners,df_preprocessed_202104042151_Boerenverstand_Ma...,../../data/best_model_202104192018.pickle


In [16]:
# opslaan van beste model properties naar csv
best_model_properties.to_csv(f'../../data/log_train/best_model_properties_{suffix_datetime}.csv', index = False, header=True)

# Feature importance

In [17]:
feature_importances = grid_search_total.best_estimator_.named_steps["clf"].feature_importances_
important_attributes = sorted(zip(feature_importances, X_train.columns), reverse=True)
important_attributes[:40]

[(0.1931905, 'relative_huishoudensmetkinderen'),
 (0.103897125, 'relative_poparbeidsongeschiktheidtotaal'),
 (0.061789, 'relative_eenpersoonshuishoudens'),
 (0.04981223, 'popk80jaarofouderrelatieveleeftijdsgroep'),
 (0.029199164, 'relative_alleenstaande_vrouwen'),
 (0.027832935, 'ouder_in_eenouderhuishouden_vrouwen'),
 (0.027375886, 'relative_popnietstedelijk'),
 (0.026788916, 'relative_huishoudenszonderkinderen'),
 (0.025175598, 'relative_mannen'),
 (0.02457151, 'relative_popmatigstedelijk'),
 (0.024438309, 'relative_poptotaleoppervlakte'),
 (0.024085274, 'popafstandtothuisartsenpraktijk'),
 (0.022022355, 'relative_ouder_in_eenouderhuishouden_mannen'),
 (0.021079514, 'ouder_in_eenouderhuishouden_mannen'),
 (0.021029627, 'relative_popbevolkingsdichtheid'),
 (0.020642133, 'popbevolkingsdichtheid'),
 (0.020444486, 'popk65tot80jaarrelatieveleeftijdsgroep'),
 (0.019716877, 'relative_popomgevingsadressendichtheid'),
 (0.019303676, 'verweduwd'),
 (0.01772483, 'relative_gescheiden'),
 (0.0174

## Code voor combineren output CSV's voor visualisatie en vergelijking

In [59]:
# ## combineren van de verschillende best model properties csv's naar één dataframe
# all_filenames = list_filenames(settings.train['LOG_PATH'], filename_str_contains='.csv')
# combined_logging = pd.concat([pd.read_csv(f"{settings.train['LOG_PATH']}{f}") for f in all_filenames ])
# combined_logging

## Code voor testen pickle file & predict

In [34]:
# # pickle file inladen voor predict
# loaded_model = get_latest_file(output_filename_str_contains='best_model_', datapath=datapath, filetype='pickle')
# # hoe moet ik deze score interpreteren?
# result = loaded_model.score(X_test, y_test)
# print(result)
# loaded_model.predict(X_test)

In [35]:
# regel om te testen of opgeslagen pickle file overeen komt met model
#grid_search_total.best_estimator_.predict(X_test)

In [ ]:
# dit is het beste model uit de grid search
#grid_search_total.best_estimator_